## Find the best SMA for the top 5 stocks
## Create a bias strategy
## Compare Drawndowns
## Pick the best 3 stocks on your portfolio.

In [4]:
def testStrategy(stock,start,end,SMA):
    df=yf.download(stock,start=start,end=end)
    data=df.Close.to_frame()
    data['returns']=np.log(data.Close.div(data.Close.shift(1)))
    data['SMA_S']=data.Close.rolling(int(SMA[0])).mean()
    data['SMA_L']=data.Close.rolling(int(SMA[1])).mean()
    data.dropna(inplace=True)
    
    data['position']=np.where(data['SMA_S']>data['SMA_L'],1,-1)
    data['strategy']=data['returns']*data.position.shift(1)
    data.dropna(inplace=True)
    ret=np.exp(data['strategy'].sum())
    std=data['strategy'].std()*np.sqrt(252)
    
    return ret,std

In [6]:
testStrategy('', '2000-01-01', '2023-05-05', (50,200))

[*********************100%%**********************]  1 of 1 completed


(72.5116281824021, 0.6151430045179632)